In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import os as os

# Cufflinks library allows direct plotting of Plotly interactive charts from Dataframes object
import cufflinks as cf
cf.set_config_file(offline=True)

# Heatmap of covariance matrix
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# scikit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)

In [ ]:
# Check working directory
os.getcwd() 

# Set working directory if necessary
# work_dir = "INSERT PATH TO FILE LOCATION"
# os.chdir(work_dir)

In [ ]:
data = pd.read_csv('./data/hjm_pca_2002-07.csv', index_col=0, sep ='\t')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
# Plot curve
data.iloc[0].iplot(title = 'Representation of a Yield Curve')

In [ ]:
# Plot all curves
data.T.iplot(title='Daily Yield Curves')

In [ ]:
diff_ = data.diff(-1)
diff_.dropna(inplace=True)

In [ ]:
diff_.tail()

In [ ]:
diff_.shape

In [ ]:
vol = np.std(diff_, axis=0) * 10000

In [ ]:
vol[:21].iplot(title='Volatility of Daily Yields', xTitle='Tenor', yTitle='Volatility (bps)', 
               color='cornflowerblue')

In [ ]:
cov_= pd.DataFrame(np.cov(diff_, rowvar=False)*252/10000, 
                   columns=diff_.columns, index=diff_.columns)

cov_.style.format("{:.4%}")

In [ ]:
# Heatmap appropirate for Covariance Matrix
fig_matrix = go.Figure(data=go.Heatmap(z=cov_, colorscale='Viridis'))
fig_matrix.update_layout(title='Covariance Matrix Heatmap')
fig_matrix.show()

In [ ]:
# 3D Surface Plot with larger dimensions
x, y = np.meshgrid(cov_.columns, cov_.index)
fig_surface = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])
fig_surface.add_trace(go.Surface(z=cov_.values, x=x, y=y, colorscale='Viridis'))

# Update layout for larger dimensions
fig_surface.update_layout(title='Covariance 3D Surface Plot (rotate)',
                          scene=dict(
                              xaxis=dict(title='X Axis'),
                              yaxis=dict(title='Y Axis'),
                              zaxis=dict(title='Z Axis'),
                          ),
                          width=800,  # Adjust width as needed
                          height=600  # Adjust height as needed
                          )

# Show the plot
fig_surface.show()

# Observation: if we remove the 0.08 tenor (where covariance peaks), we are likely to have better behavour of Covariance Matrix

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(cov_)

# Sort values (good practice)
idx = eigenvalues.argsort()[::-1]   
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:,idx]

# Format into DataFrame 
df_eigval = pd.DataFrame({"Eigenvalues": eigenvalues})

In [ ]:
df_eigval.head()

In [ ]:
# Work out explained proportion 
df_eigval["Var Explained"] = df_eigval["Eigenvalues"] / np.sum(df_eigval["Eigenvalues"])
df_eigval = df_eigval[:8]

#Format as percentage
df_eigval.style.format({"Var Explained": "{:.2%}"})

In [ ]:
(df_eigval['Var Explained'][:5]*100).iplot(kind='bar', title='Percentage Variance Explained', 
                                            color='cornflowerblue')

In [ ]:
# Subsume first 3 components into a dataframe
pcadf = pd.DataFrame(eigenvectors[:,0:3], columns=['e1','e2','e3'], index=data.columns)
pcadf[:10]

In [ ]:
pcadf.iplot(title='Principal Components for Forward Curve (HJM Lecture) UNSCALED', secondary_y='e1', secondary_y_title='e1 or PC1', 
            yTitle='change in yield (bps)')

In [ ]:
# Import Curve Data
df = pd.read_excel("./data/gilts_spot_1970-2015.xlsx", index_col=0, header=3, sheet_name="4. spot curve", skiprows=[4])

# IMPORTANT DATA PROCESSING
# Limit curve to 10.5Y tenor.  Semi-annual increments give 20 columns
# Tenor 0.5Y would have given a lot missed values, so the entire monthy curve would've been eliminated

df = df.iloc[:, 1:21]

df.head()

In [ ]:
df = df.dropna(how="any")
df.shape

In [ ]:
# StandardScaler() by defaults normalises data -- computes Z-scores by column
scaler = StandardScaler()
scaler.fit(df)

df1 = pd.DataFrame(scaler.transform(df))
df1.head()

In [ ]:
cov_array = np.cov(df1, rowvar=False)
# cov_df1 = pd.DataFrame(cov_array) #, index=range(1,21), columns=range(1,21))

cov_df1 = pd.DataFrame(cov_array, columns=df.columns, index=df.columns)
cov_df1 .style.format("{:.4}")

In [ ]:
# 3D Surface Plot with larger dimensions
x, y = np.meshgrid(cov_df1.columns, cov_df1.index)
fig_surface = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])
fig_surface.add_trace(go.Surface(z=cov_df1.values, x=x, y=y, colorscale='Viridis'))

# Update layout for larger dimensions
fig_surface.update_layout(title='Covariance 3D Surface Plot (rotate)',
                          scene=dict(
                              xaxis=dict(title='X Axis'),
                              yaxis=dict(title='Y Axis'),
                              zaxis=dict(title='Z Axis'),
                          ),
                          width=800,  # Adjust width as needed
                          height=600  # Adjust height as needed
                          )

# Show the plot
fig_surface.show()

# Observation: we have ended up with very robust covariance matrix, devoid of noise. High correlations.

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(cov_array)

# Sort values (good practice)
idx = eigenvalues.argsort()[::-1]   
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:,idx]

# Format into DataFrame (but we output array type below -- to show how small eigenvalues became)
df1_eigval = pd.DataFrame({"Eigenvalues": eigenvalues}) #, index=range(1,21))

In [ ]:
eigenvalues

In [ ]:
# Format into a DataFrame 
df1_eigvec = pd.DataFrame(eigenvectors) #, index=range(1,21))

eigenvectors[:,0] # Only PC1 is of relevance

In [ ]:
# Work out explained proportion 
df1_eigval["Var Explained"] = df_eigval["Eigenvalues"] / np.sum(df_eigval["Eigenvalues"])
df1_eigval = df_eigval[:5]

#Format as percentage
df1_eigval.style.format({"Var Explained": "{:.2%}"})

In [ ]:
(df1_eigval['Var Explained'][:5]*100).iplot(kind='bar', 
                                             title='Percentage Variance Explained', 
                                             color='cornflowerblue')

In [ ]:
# Subsume first 3 components into a dataframe
pcdf = pd.DataFrame(eigenvectors[:,0:3], columns=['e1','e2','e3'], index=df.columns)
pcdf[:10]

In [ ]:
pcdf.iplot(title='Principal Components for Gilts Curve (unscaled)', secondary_y='e1', secondary_y_title='PC1 or e1')

In [ ]:
# Wrap two main stages into a pipeline
pipe = Pipeline([("scaler", StandardScaler()), ("pca", PCA())]) 
pipe.fit(df)

In [ ]:
# eigenvectors
pipe['pca'].components_[0]

In [ ]:
# eigenvalues, reference here to eigenvalues being canonical variances
pipe['pca'].explained_variance_

In [ ]:
# explained variance ratio is R^2 statistic, eg 98.89% for our PC1 below
pipe['pca'].explained_variance_ratio_

In [ ]:
df1_eigval = pd.DataFrame({'Eigenvalues': pipe['pca'].explained_variance_,
                    'Var Explained': pipe['pca'].explained_variance_ratio_})
df1_eigval = df1_eigval[:5]

#Format as percentage
df1_eigval.style.format({"Var Explained": "{:.2%}"})

In [ ]:
# Dot product below 'projects' principal components, onto the scaled dataframe df1 (tenors x curves)

df1_projections = df1.dot(eigenvectors)  # all 20 eigenvectors preserved for dot product to work, and return Nrows the same as data

df1_projections.index = df.index
df1_projections.head(10)

In [ ]:
#Check dimensions
df1_projections.shape

In [ ]:
# Plot all 
df1_projections.iplot(title='Projections')

# data.T.iplot(title='Quasi curves') this plot not very useful, it will show that beyond 2nd-3rd column there is no curve information in the projection dataset

In [ ]:
df1_projections3 = df1.dot(eigenvectors[:, 0:3])  # only 3 eigenvectors are preserved
df1_projections3.index = df.index

df1_projections3.shape

In [ ]:
df1_projections3.head(10)

In [ ]:
level = pd.DataFrame({'10Y': df[2.0],
                  'pc1_projection': df1_projections[0]})
level.head()

In [ ]:
level.iplot(title='PC1 Projection vs 10Y Yield', secondary_y='pc1_projection')

In [ ]:
# Calculate 10Y-2Y, typical measure of slope
slope = pd.DataFrame(df)
slope = slope[[2,10]]
slope['slope'] = slope[10] - slope[2]
slope['pc2_projection'] = - df1_projections[1] # here e2 demonstrated its indifference to sign and got inverted
slope.head()

In [ ]:
slope[['slope', 'pc2_projection']].iplot(title='PC2 Projection vs 10Y-2Y Slope', secondary_y='pc2_projection')

In [ ]:
# Verify the correlation
np.corrcoef(-df1_projections[1], slope['slope'])

In [ ]:
# Sample dataset (10 rows x 3 columns) representing interest rates (3 tenors)
rates_data = np.array([
    [0.038, 0.040, 0.045],
    [0.041, 0.042, 0.046],
    [0.044, 0.046, 0.048],
    [0.049, 0.048, 0.049],
    [0.046, 0.043, 0.047],
    [0.045, 0.044, 0.048],
    [0.047, 0.049, 0.046],
    [0.045, 0.047, 0.044],
    [0.039, 0.041, 0.050],
    [0.040, 0.043, 0.048]
])

# Example single eigenvector (1 x 3 dimensions)
eigenvector = np.array([[0.1, 0.2, 0.3]])

# Perform dot product
projected_data = rates_data.dot(eigenvector.T)

print("\nEigenvector:")
print(eigenvector)

print("\nProjected Data: alike to 10 daily values of 1 projection")
print(projected_data)

In [ ]:
# Now let's have 3 eigenvectors
eigenvectors = np.array([
    [0.1, 0.2, 0.3],
    [0.2, 0.3, 0.1],
    [0.3, 0.1, 0.2]
])

# Dot product with all 3 eigenvectors
projected_data = rates_data.dot(eigenvectors.T)

print("\nProjected Data: now we have 10 daily values of 3 projections")
print(projected_data)